<a href="https://colab.research.google.com/github/Tanu-N-Prabhu/Python/blob/master/Machine%20Learning%20Interview%20Prep%20Questions/Unsupervised%20Learning%20Algorithms/Association%20Rules/Frequent%20Pattern%20Growth/fp_growth_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FP-Growth Algorithm from Scratch (Pure Python)

FP-Growth (Frequent Pattern Growth) is a high-performance algorithm for discovering frequent itemsets without candidate generation.

- Uses a **compressed prefix tree** (FP-Tree)
- Avoids scanning database repeatedly (unlike Apriori)
- Ideal for **large-scale transaction data**


## Sample Transactions

In [1]:
transactions = [
    ['milk', 'bread', 'butter'],
    ['bread', 'butter'],
    ['milk', 'bread'],
    ['milk', 'bread', 'butter', 'jam'],
    ['bread', 'jam'],
    ['milk', 'bread', 'jam']
]

## FP-Growth Components
### Count Item Frequencies

In [2]:
from collections import defaultdict

def get_item_counts(transactions):
    item_counts = defaultdict(int)
    for tx in transactions:
        for item in tx:
            item_counts[item] += 1
    return item_counts

### Build FP-Tree Node Class


In [3]:
class FPTreeNode:
    def __init__(self, item, parent):
        self.item = item
        self.count = 1
        self.parent = parent
        self.children = {}
        self.link = None  # Used to create header table chains

    def increment(self):
        self.count += 1

### Build FP-Tree


In [4]:
def build_fp_tree(transactions, min_support):
    item_counts = get_item_counts(transactions)
    item_counts = {item: count for item, count in item_counts.items() if count >= min_support}

    # Header Table: item -> [support, first node]
    header_table = {item: [count, None] for item, count in item_counts.items()}
    root = FPTreeNode(None, None)

    for tx in transactions:
        # Filter and sort items in transaction
        filtered_tx = [item for item in tx if item in item_counts]
        sorted_tx = sorted(filtered_tx, key=lambda i: item_counts[i], reverse=True)

        # Insert into tree
        current_node = root
        for item in sorted_tx:
            if item in current_node.children:
                current_node.children[item].increment()
            else:
                new_node = FPTreeNode(item, current_node)
                current_node.children[item] = new_node

                # Link to header table
                if header_table[item][1] is None:
                    header_table[item][1] = new_node
                else:
                    node = header_table[item][1]
                    while node.link:
                        node = node.link
                    node.link = new_node

            current_node = current_node.children[item]

    return root, header_table

### Mine Frequent Patterns



In [5]:
def ascend_fp_tree(node):
    path = []
    while node.parent and node.parent.item is not None:
        node = node.parent
        path.append(node.item)
    return path[::-1]  # reverse path

def find_prefix_paths(base_pat, node):
    paths = []
    while node:
        path = ascend_fp_tree(node)
        if path:
            paths.append((path, node.count))
        node = node.link
    return paths

## Recursive Mining

In [6]:
def mine_fp_tree(header_table, min_support, prefix, frequent_itemsets):
    items = sorted(header_table.items(), key=lambda i: i[1][0])  # Ascending frequency

    for base_item, (support, node) in items:
        new_freq_set = prefix + [base_item]
        frequent_itemsets[tuple(new_freq_set)] = support

        conditional_patterns = find_prefix_paths(base_item, node)

        # Build conditional transactions
        conditional_transactions = []
        for path, count in conditional_patterns:
            conditional_transactions.extend([path] * count)

        # Build conditional FP-Tree
        if conditional_transactions:
            conditional_tree, new_header = build_fp_tree(conditional_transactions, min_support)
            if new_header:
                mine_fp_tree(new_header, min_support, new_freq_set, frequent_itemsets)

## Run FP-Growth

In [7]:
min_support = 3  # absolute count
fp_tree, header_table = build_fp_tree(transactions, min_support)
frequent_itemsets = {}
mine_fp_tree(header_table, min_support, [], frequent_itemsets)

print("Frequent Itemsets:")
for itemset, count in frequent_itemsets.items():
    print(f"{itemset}: {count}")

Frequent Itemsets:
('butter',): 3
('butter', 'bread'): 3
('jam',): 3
('jam', 'bread'): 3
('milk',): 4
('milk', 'bread'): 4
('bread',): 6
